In [1]:
import mysql.connector
from mysql.connector import Error
import json
import re
import pandas as pd
from datetime import datetime
from DBHelper import CDBHelper
from ClickUpDB import CClickUpDB
import os

In [2]:
list_id = "901600183071"  # Replace with your actual ListID
start_date = "01-08-2024"  # Replace with your desired start date
end_date = "01-09-2024"    # Replace with your desired end date

# Fetch tasks based on the criteria
tasks = CClickUpDB.MSGetTasksByListID(list_id, start_date, end_date)
print(len(tasks))

Successfully connected to MySQL server
Retrieved 54 task(s) from ListID 901600183071 between 01-08-2024 and 01-09-2024.
54


In [4]:
def MSCreateDataframe(lsTasks):
    # Creating a DataFrame from the list of tasks
    
    df = pd.DataFrame(lsTasks)
    # Parsing JSON strings in 'TaskPriority' and 'TaskAssigneesList'
    df['TaskPriority'] = df['TaskPriority'].apply(lambda x: json.loads(x) if x != 'null' else None)
    df['TaskAssigneesList'] = df['TaskAssigneesList'].apply(json.loads)
    df['TaskScore'] = df.apply(CClickUpDB.calculate_task_score, include_toughness=False, axis=1)
    
    df = df[df['EstimatedTime'].apply(CClickUpDB.isEstimatedTimeProvided)]

    # Convert 'TaskStartDate' to datetime for sorting
    df['TaskStartDate'] = pd.to_datetime(df['TaskStartDate'], format='%d-%m-%Y')

    # Sort tasks by 'TaskStartDate' ascending, then by 'TaskScore' ascending within the same date
    df = df.sort_values(by=['TaskStartDate', 'TaskScore'])
    # Drop columns except the specified list
    columns_to_keep = ['ListName', 'TaskID', 'TaskSubject', 'TaskStartDate', 'TaskDueDate', 
                       'EstimatedTime', 'TaskPriority', 'TaskAssigneesList', 
                       'TaskIsMilestone', 'TaskIntensity', 'TaskScore']
    df = df[columns_to_keep]
    # Creating employee-wise DataFrames
    employee_tasks = {}
    for _, row in df.iterrows():
        print(row)
        assignees = row['TaskAssigneesList']
        if assignees:
            for assignee in assignees:
                username = assignee['username']
                if username not in employee_tasks:
                    employee_tasks[username] = []
                employee_tasks[username].append(row)

    # # Convert the lists into DataFrames
    employee_dataframes = {username: pd.DataFrame(tasks) for username, tasks in employee_tasks.items()}

    # Directory to save the Excel files
    output_directory = r"Data/"
    os.makedirs(output_directory, exist_ok=True)  # Create the directory if it doesn't exist

    # Save employee-wise DataFrames to Excel files
    for username, df in employee_dataframes.items():
        # Create a filename based on the username
        filename = f"{username.replace(' ', '_')}_tasks.xlsx"
        # Construct the full path
        file_path = os.path.join(output_directory, filename)
        # Save the DataFrame to an Excel file
        df.to_excel(file_path, index=False)

    print("Employee-wise task details have been saved to the 'Data/' directory.")
    # print(df)

MSCreateDataframe(tasks)

ListName                                                       ERPNext
TaskID                                                       86cw76fjj
TaskSubject           Mitul - Compare Odoo vs Zoho vs ERPNext Solution
TaskStartDate                                      2024-08-14 00:00:00
TaskDueDate                                                 21-08-2024
EstimatedTime         {"hrs": 0, "mins": 30, "time_estimate": 1800000}
TaskPriority         {'id': '1', 'color': '#f50000', 'priority': 'u...
TaskAssigneesList    [{'id': 67390920, 'color': '', 'email': 'mitul...
TaskIsMilestone                                                      0
TaskIntensity                                                        1
TaskScore                                                            3
Name: 6, dtype: object
ListName                                                       ERPNext
TaskID                                                       86cw7wkbw
TaskSubject                                    Restore